In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model.safetensors.index.json
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00001-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/config.json
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00003-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00002-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00007-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/README.md
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00008-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/tokenizer.json
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/tokenizer_config.json
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00005-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/model-00006-of-00008.safetensors
/kaggle/input/gemma-2/transformers/gemma-2-9b/2/special_tokens_map.json
/kaggle/input/gemma-2/transformer

In [2]:
import kagglehub

# Download latest version
path = kagglehub.model_download("google/gemma-2/transformers/gemma-2-9b")

print("Path to model files:", path)

Path to model files: /kaggle/input/gemma-2/transformers/gemma-2-9b/2


In [3]:
import random
import gc
import torch
import pandas as pd
from collections import Counter
from typing import List, Union
import numpy as np
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

class PerplexityCalculator:
    def __init__(self, model_path: str, load_in_8bit: bool = False, device_map: str = 'auto'):
        # Charger le tokenizer depuis le modèle spécifié, avec option pour exécuter du code distant
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            trust_remote_code=True
        )

        # Si load_in_8bit est activé, configurer le modèle pour le chargement en 8 bits
        if load_in_8bit:
            quantization_config = transformers.BitsAndBytesConfig(load_in_8bit=True)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_path,
                quantization_config=quantization_config,
                device_map=device_map,
                trust_remote_code=True
            )
        else:
            # Charger le modèle normalement avec le type de données approprié
            self.model = AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map=device_map,
                trust_remote_code=True
            )

        # Fonction de perte pour le calcul de la perplexité
        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
        self.model.eval()  # Mettre le modèle en mode évaluation

    def get_perplexity(self, input_texts: Union[str, List[str]], batch_size: int = 8) -> Union[float, List[float]]:
        # Vérifier si l'entrée est une chaîne unique ou une liste
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts
        loss_list = []

        # Traiter les textes en lots pour le calcul de la perplexité
        for i in range(0, len(input_texts), batch_size):
            batch_texts = input_texts[i:i + batch_size]  # Créer un lot de textes
            with torch.no_grad():  # Désactiver le calcul des gradients
                model_inputs = self.tokenizer(
                    batch_texts,
                    return_tensors='pt',  # Retourner des tenseurs PyTorch
                    padding=True,  # Activer le padding pour les entrées de longueur variable
                    truncation=True,  # Troncature des textes trop longs
                    max_length=512,  # Longueur maximale des séquences
                    add_special_tokens=True,  # Ajouter des tokens spéciaux si nécessaire
                ).to(self.model.device)

                # Obtenir les logits du modèle
                output = self.model(**model_inputs)
                logits = output.logits

                # Préparer les labels pour le calcul de la perte
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = model_inputs['input_ids'][..., 1:].contiguous()

                # Calculer la perte
                loss = self.loss_fct(
                    shift_logits.view(-1, shift_logits.size(-1)),
                    shift_labels.view(-1)
                )

                # Calculer la perte par batch
                batch_losses = loss.view(len(batch_texts), -1).sum(dim=1) / model_inputs['attention_mask'].sum(dim=1)
                loss_list.extend(batch_losses.cpu().tolist())  # Ajouter les pertes calculées à la liste

        # Calculer la perplexité à partir des pertes
        ppl = [np.exp(i) for i in loss_list]
        return ppl[0] if single_input else ppl  # Retourner la perplexité pour une entrée unique ou en liste

    def clear_memory(self) -> None:
        # Libérer la mémoire en supprimant le modèle et le tokenizer
        del self.model
        del self.tokenizer
        gc.collect()  # Forcer le ramassage des ordures pour libérer la mémoire

In [4]:
import random
import gc

# Définir la classe GeneticAlgorithm
class GeneticAlgorithm:
    def __init__(self, random_state, population_size, generations, mutation_rate, elite_rate):
        random.seed(random_state)  # Initialiser le générateur aléatoire avec un état donné
        self.population_size = population_size  # Taille de la population
        self.generations = generations  # Nombre de générations à parcourir
        self.mutation_rate = mutation_rate  # Taux de mutation
        self.elite_rate = elite_rate  # Taux d'élitisme

    def order_crossover(self, parent1, parent2):
        # Effectuer le croisement de type "order" entre deux parents
        start, end = sorted(random.sample(range(len(parent1)), 2))  # Choisir deux indices aléatoires
        child = [None] * len(parent1)  # Initialiser l'enfant avec des None
        child[start:end+1] = parent1[start:end+1]  # Copier la section du premier parent
        remaining = parent2.copy()  # Copier le deuxième parent
        for used_word in child:
            if used_word is not None:
                remaining.remove(used_word)  # Retirer les mots utilisés de l'autre parent
        j = 0
        for i in range(len(child)):
            if child[i] is None:
                child[i] = remaining[j]  # Remplir l'enfant avec les mots restants
                j += 1
        return child

    def mutate(self, individual):
        # Appliquer une mutation à un individu
        if random.random() < self.mutation_rate:  # Vérifier si la mutation doit avoir lieu
            i, j = random.sample(range(len(individual)), 2)  # Sélectionner deux indices aléatoires
            individual[i], individual[j] = individual[j], individual[i]  # Échanger les mots
        return individual

    def gradient_descent(self, sequence, scorer, learning_rate=0.01, max_iterations=20):
        # Appliquer la descente de gradient pour améliorer une séquence
        current_sequence = sequence[:]
        best_sequence = sequence[:]
        best_perplexity = scorer.get_perplexity(' '.join(best_sequence))  # Calculer la perplexité de la meilleure séquence

        for _ in range(max_iterations):
            neighbor = current_sequence[:]  # Créer un voisin en copiant la séquence actuelle
            i, j = random.sample(range(len(neighbor)), 2)  # Sélectionner deux indices aléatoires
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]  # Échanger les mots pour créer un voisin
            neighbor_perplexity = scorer.get_perplexity(' '.join(neighbor))  # Calculer la perplexité du voisin
            delta = neighbor_perplexity - best_perplexity  # Calculer le changement de perplexité
            if delta < 0:  # Si le voisin est meilleur
                current_sequence = neighbor[:]  # Mettre à jour la séquence actuelle
                if neighbor_perplexity < best_perplexity:  # Si le voisin est meilleur que le meilleur
                    best_sequence = neighbor[:]  # Mettre à jour la meilleure séquence
                    best_perplexity = neighbor_perplexity  # Mettre à jour la meilleure perplexité

        return best_sequence  # Retourner la meilleure séquence trouvée

    def solve(self, text, scorer):
        # Résoudre le problème en utilisant l'algorithme génétique
        words = text.split()  # Diviser le texte en mots
        word_N = len(words)  # Compter le nombre de mots
        # Initialiser la population avec des permutations aléatoires des mots
        population = [random.sample(words, word_N) for _ in range(self.population_size)]
        log_energies = []  # Liste pour suivre les perplexités minimales de chaque génération

        for generation in range(self.generations):
            # Évaluer la fitness de chaque individu dans la population
            fitness_scores = [scorer.get_perplexity(' '.join(ind)) for ind in population]
            min_perplexity = min(fitness_scores)  # Trouver la perplexité minimale
            log_energies.append(min_perplexity)  # Enregistrer la perplexité minimale
            print(f"Generation {generation + 1}/{self.generations} - Best Perplexity: {min_perplexity}")

            # Sélectionner les meilleurs individus (élites)
            elite_indices = sorted(range(len(fitness_scores)),
                                   key=lambda k: fitness_scores[k])[:int(self.population_size * self.elite_rate)]
            new_population = [population[i] for i in elite_indices]  # Créer une nouvelle population avec les élites

            # Compléter la nouvelle population jusqu'à la taille d'origine
            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(population, 2)  # Sélectionner deux parents aléatoires
                child = self.order_crossover(parent1, parent2)  # Créer un enfant par croisement
                new_population.append(child)  # Ajouter l'enfant à la nouvelle population

            # Appliquer la mutation à tous les individus de la nouvelle population
            for i in range(len(new_population)):
                new_population[i] = self.mutate(new_population[i])

            # Appliquer la descente de gradient aux élites
            for i in range(int(len(new_population) * self.elite_rate)):
                new_population[i] = self.gradient_descent(new_population[i], scorer)

            population = new_population  # Mettre à jour la population

        # Évaluer la fitness finale de la population
        fitness_scores = [scorer.get_perplexity(' '.join(ind)) for ind in population]
        best_individual = population[np.argmin(fitness_scores)]  # Trouver le meilleur individu
        best_energy = min(fitness_scores)  # Meilleure perplexité
        print("\nGenetic Algorithm Optimization Complete")
        print(f"Best Perplexity: {best_energy}")

        return ' '.join(best_individual), best_energy, log_energies  # Retourner le meilleur texte, sa perplexité et les enregistrements

In [ ]:
# Fonction pour optimiser des séquences avec l'algorithme génétique (GA)
def optimize_sequences_with_ga(batch_size=5):
    # Charger le fichier de soumission d'exemple
    sample_submission = pd.read_csv("/kaggle/input/santa1/sample_submission.csv")
    results = []  # Liste pour stocker les résultats

    # Initialiser l'algorithme génétique avec des paramètres spécifiques
    ga = GeneticAlgorithm(
        random_state=42,  # État aléatoire pour la reproductibilité
        population_size=10,  # Taille de la population pour GA
        generations=10,  # Nombre de générations d'optimisation
        mutation_rate=0.1,  # Taux de mutation
        elite_rate=0.2  # Taux d'élitisme
    )

    # Calculer le nombre de lots à traiter
    num_batches = (len(sample_submission) + batch_size - 1) // batch_size

    # Traiter chaque lot de données
    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size  # Index de début pour le lot
        batch_end = min((batch_idx + 1) * batch_size, len(sample_submission))  # Index de fin pour le lot
        batch_data = sample_submission.iloc[batch_start:batch_end]  # Extraire le lot de données

        print(f"\nProcessing batch {batch_idx + 1}/{num_batches}")  # Afficher le numéro de lot en cours de traitement

        try:
            # Initialiser le calculateur de perplexité avec le modèle spécifié
            scorer = PerplexityCalculator(
                model_path='/kaggle/input/gemma-2/transformers/gemma-2-9b/2',
                load_in_8bit=False
            )

            # Traiter chaque ligne dans le lot
            for idx, row in batch_data.iterrows():
                specific_solution = pd.DataFrame({'id': [row['id']], 'text': [row['text']]})  # Créer un DataFrame pour la solution spécifique
                text_sequence = row['text'].split()  # Diviser le texte en mots

                # Optimiser le texte en utilisant l'algorithme génétique
                optimized_text, final_score, log_energies = ga.solve(row['text'], scorer)

                # Afficher le résultat final pour chaque ID
                print(f"ID: {row['id']}, Final Perplexity: {final_score}")
                results.append({'id': row['id'], 'text': optimized_text})  # Ajouter le résultat à la liste

            # Enregistrer les résultats temporaires du lot dans un fichier CSV
            temp_df = pd.DataFrame(results)
            temp_df.to_csv(f"submission_temp_batch_{batch_idx+1}.csv", index=False)

        except Exception as e:
            # Gérer les erreurs durant le traitement des lots
            print(f"Error processing batch {batch_idx + 1}: {str(e)}")
            # Si une erreur se produit, ajouter les textes originaux aux résultats
            for idx, row in batch_data.iterrows():
                results.append({'id': row['id'], 'text': row['text']})
                print("results >>>>>>>>>>>>>>>>")
                print({'id': row['id'], 'text': row['text']})  # Afficher le résultat original
                print("------------------------------------------------")

    # Créer le DataFrame final avec tous les résultats et l'enregistrer dans un fichier CSV
    submission = pd.DataFrame(results)
    submission.to_csv("submission.csv", index=False)
    return submission  # Retourner le DataFrame de soumission final

if __name__ == "__main__":
    print("Starting GA optimization...")
    final_submission = optimize_sequences_with_ga()  # Lancer l'optimisation
    print("Optimization with GA completed!")

Starting GA optimization...

Processing batch 1/2


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Generation 1/10 - Best Perplexity: 850.8240325202685
Generation 2/10 - Best Perplexity: 621.4654194025813
Generation 3/10 - Best Perplexity: 610.160500583438
Generation 4/10 - Best Perplexity: 374.05994496126846
Generation 5/10 - Best Perplexity: 374.05994496126846
Generation 6/10 - Best Perplexity: 350.0905961076616
Generation 7/10 - Best Perplexity: 336.633887513654
Generation 8/10 - Best Perplexity: 336.633887513654
Generation 9/10 - Best Perplexity: 317.99778895332923
Generation 10/10 - Best Perplexity: 317.99778895332923

Genetic Algorithm Optimization Complete
Best Perplexity: 317.99778895332923
ID: 0, Final Perplexity: 317.99778895332923
Generation 1/10 - Best Perplexity: 3136.3788011047473
Generation 2/10 - Best Perplexity: 2321.0732058825593
Generation 3/10 - Best Perplexity: 2002.2406771787246
Generation 4/10 - Best Perplexity: 1554.5083846251587
Generation 5/10 - Best Perplexity: 919.6807368245869
Generation 6/10 - Best Perplexity: 804.781899278158
Generation 7/10 - Best Per

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Après l'optimisation de l'algorithme génétique, il est à noter que le batch 2, qui contient la séquence 6, s'est arrêté en raison de contraintes de ressources. Pour surmonter ce problème, nous avons décidé d'exécuter le runner de manière autonome dans la cellule suivante.

In [8]:
# Function to optimize sequences with GA for second batch
def optimize_sequences_with_ga(batch_size=5):
    sample_submission = pd.read_csv("/kaggle/input/santa1/sample_submission.csv")
    results = []

    ga = GeneticAlgorithm(
        random_state=42,
        population_size=10,
        generations=10,
        mutation_rate=0.1,
        elite_rate=0.2
    )

    # Calculate the total number of batches
    num_batches = (len(sample_submission) + batch_size - 1) // batch_size

    # Set batch_idx to 1 for the second batch (index starts at 0)
    batch_idx = 1  # Only process the second batch
    
    # Calculate the start and end index for the second batch
    batch_start = batch_idx * batch_size
    batch_end = min((batch_idx + 1) * batch_size, len(sample_submission))
    batch_data = sample_submission.iloc[batch_start:batch_end]

    print(f"\nProcessing batch {batch_idx + 1}/{num_batches}")

    try:
        scorer = PerplexityCalculator(
            model_path='/kaggle/input/gemma-2/transformers/gemma-2-9b/2',
            load_in_8bit=False
        )

        for idx, row in batch_data.iterrows():
            specific_solution = pd.DataFrame({'id': [row['id']], 'text': [row['text']]})
            text_sequence = row['text'].split()

            optimized_text, final_score, log_energies = ga.solve(row['text'], scorer)

            print(f"ID: {row['id']}, Final Perplexity: {final_score}")
            results.append({'id': row['id'], 'text': optimized_text})

        temp_df = pd.DataFrame(results)
        temp_df.to_csv(f"submission_temp_batch_{batch_idx+1}.csv", index=False)

    except Exception as e:
        print(f"Error processing batch {batch_idx + 1}: {str(e)}")
        for idx, row in batch_data.iterrows():
            results.append({'id': row['id'], 'text': row['text']})
            print("results >>>>>>>>>>>>>>>>")
            print({'id': row['id'], 'text': row['text']})
            print("------------------------------------------------")

    # Final submission for the second batch
    submission = pd.DataFrame(results)
    submission.to_csv("submission.csv", index=False)
    return submission

if __name__ == "__main__":
    print("Starting GA optimization...")
    final_submission = optimize_sequences_with_ga()
    print("Optimization with GA completed!")


Starting GA optimization...

Processing batch 2/2


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Generation 1/10 - Best Perplexity: 786.1271512208434
Generation 2/10 - Best Perplexity: 699.2579777721731
Generation 3/10 - Best Perplexity: 608.8964823056507
Generation 4/10 - Best Perplexity: 571.1810975204792
Generation 5/10 - Best Perplexity: 529.6956665489097
Generation 6/10 - Best Perplexity: 520.519328864813
Generation 7/10 - Best Perplexity: 485.67502259252456
Generation 8/10 - Best Perplexity: 453.0076893306675
Generation 9/10 - Best Perplexity: 424.7089026586955
Generation 10/10 - Best Perplexity: 413.866201188035

Genetic Algorithm Optimization Complete
Best Perplexity: 404.86913074332483
ID: 5, Final Perplexity: 404.86913074332483
Optimization with GA completed!
